In [2]:
import csv
with open('hrishipara_financial_diaries.csv') as var_name:
    reader=csv.reader(var_name)
    A = [row for row in reader]
import pingouin as pg
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import font_manager as fm
from matplotlib import cm
pd.set_option('display.max_rows', None)
df = pd.DataFrame(A)
new_header = df.iloc[0] #grab the first row for the header
df = df[1:] #take the data less the header row
df.columns = new_header #set the header row as the df header
df['code']=df['code'].str.upper()
df['description']=df['description'].str.lower()
df["takaout"] = pd.to_numeric(df["takaout"])
df["takain"] = pd.to_numeric(df["takain"])
df['date']=pd.to_datetime(df.date)
df.drop(df.loc[df['date'] < '2020-01-01'].index, inplace=True)
df.drop(df.loc[df['date'] > '2020-12-31'].index, inplace=True)
df.head()

/Users/inouenoritake/opt/miniconda3/lib/python3.9/site-packages/outdated/utils.py:14: OutdatedPackageWarning: The package pingouin is out of date. Your version is 0.3.11, the latest is 0.3.12.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


,date,ref,code,class,description,subclass,tag,takain,takaout,details,notes,flow,type,quartile
772088,2020-01-01,no rec,01KIF,IWA,earned income,shop labour,no rec,36,0,onion and potato shop,no rec,in,income,4
772089,2020-01-01,no rec,01KIF,EFO,food costs,bakery,no rec,0,-10,bread,no rec,out,expenditure,4
772090,2020-01-01,no rec,01KIF,EFO,food costs,herbs and spices,no rec,0,-5,chilli,no rec,out,expenditure,4
772091,2020-01-01,no rec,01KIF,EOT,other expenditure,misc living expenses,no rec,0,-10,daughter,no rec,out-t,expenditure,4
772092,2020-01-01,no rec,01KIF,EFO,food costs,fish,no rec,0,-60,fish,no rec,out,expenditure,4


In [28]:
df_takaout=df.drop(df[(df.takaout==0)].index)
Expense=['snacks, treats, stimulants', 'transport costs', 'other expenditure', 'food costs',
         'toiletries purchased', 'communications',
        'fuel costs', 'home maintenance', 'clothes, cloth',
        'vehicle repair costs', 'rent paid', 'personal grooming',
        'healthcare costs',  
        'farm inputs', 'education costs', 'home construction',
        'gifts to outsiders', 'utilities cost', 'fees paid',
         'work tools and materials', 'jewellery purchases',
        'payment to others']

#'opening balance', 'savings deposited','loan repayment made','business asset purchase',
#,'shop stock purchases','wages paid out','emoney stock purchases',"jasim's secret sim transactions"'newspaper stocks',

In [8]:
df_takaout['description'].unique()

array(['food costs', 'other expenditure', 'transport costs',
       'snacks, treats, stimulants', 'home maintenance',
       'toiletries purchased', 'wages paid out', 'utilities cost',
       'savings deposited', 'loan repayment made', 'education costs',
       'clothes, cloth', 'shop stock purchases', 'communications',
       'personal grooming', 'work tools and materials', 'fuel costs',
       'healthcare costs', 'payment to others', 'fees paid',
       'gifts to outsiders', 'farm inputs', 'newspaper stocks',
       'vehicle repair costs', 'home construction',
       'recycling stock purchases', 'repayment made to diary project',
       'rent paid', 'business asset purchase', 'jewellery purchases',
       'loan or howlat given', 'real estate purchases'], dtype=object)

In [29]:
set(Expense)-set(['food costs', 'other expenditure', 'transport costs',
       'snacks, treats, stimulants', 'home maintenance',
       'toiletries purchased', 'wages paid out', 'utilities cost',
       'savings deposited', 'loan repayment made', 'education costs',
       'clothes, cloth', 'shop stock purchases', 'communications',
       'personal grooming', 'work tools and materials', 'fuel costs',
       'healthcare costs', 'payment to others', 'fees paid',
       'gifts to outsiders', 'farm inputs', 'newspaper stocks',
       'vehicle repair costs', 'home construction',
       'recycling stock purchases', 'repayment made to diary project',
       'rent paid', 'business asset purchase', 'jewellery purchases',
       'loan or howlat given', 'real estate purchases'])

set()

In [128]:
df_takain=df.drop(df[(df.takain==0)].index)
df_takain['description'].unique()

array(['earned income', 'payment from diary project',
       'hairdressing income', 'gifts from outsiders', 'shop sales',
       'farm sales', 'newspaper sales', 'vehicle repair and sales income',
       'dressmaking income', 'rickshaw income', 'musical income',
       'home or street snack sales', 'boatman income', 'fishing income',
       'income as salesman', 'payments received', 'rent income',
       'washing and laundry income', 'overseas remittance',
       'loan repayment received', 'shoe sales and repair',
       'loan or howlat taken', 'recycling income', 'sales of assets',
       'savings withdrawal taken', 'other income', 'welfare payments',
       'coaching income', 'faith healing income'], dtype=object)

In [130]:
Income=['earned income', 'payment from diary project',
       'hairdressing income', 'gifts from outsiders', 'shop sales',
       'farm sales', 'newspaper sales', 'vehicle repair and sales income',
       'dressmaking income', 'rickshaw income', 'musical income',
       'home or street snack sales', 'boatman income', 'fishing income',
       'income as salesman', 'payments received', 'rent income',
       'washing and laundry income', 'overseas remittance',
       'loan repayment received', 'shoe sales and repair',
       'loan or howlat taken', 'recycling income', 'sales of assets',
       'savings withdrawal taken', 'other income', 'welfare payments',
       'coaching income', 'faith healing income']

In [131]:
Income_sum={}
for i in Income:    
    x=df_takain[df_takain['description']==i].sum()['takain']
    x=abs(x)
    Income_sum[i]=x
Income_q=sorted(Income_sum.values())
Income_labels=[]
for x in Income_q:
    for name, number in Income_sum.items():
        if x == number:
            Income_labels.append(name)
%matplotlib

def my_autopct(pct):
    return ('%.2f' % pct) if pct > 1.765 else ''
colors= cm.rainbow(np.arange(len(Income_q))/len(Income_q))
sizes=(np.array(Income_q)/sum(Income_q))*100
labels = [f'{l}, {s:0.1f}%' for l, s in zip(Income_labels, sizes)]
fig, ax = plt.subplots(figsize=(10,5))
ax.pie(Income_q,labels=labels,colors=colors,startangle=90,autopct=my_autopct,rotatelabels=False,textprops={"size": 10})
handles, labels = ax.get_legend_handles_labels()


plt.legend(handles[::-1], labels[::-1],bbox_to_anchor=(1.1, 1.05))
plt.show()

Using matplotlib backend: MacOSX


In [30]:
Expense_sum={}
for i in Expense:    
    x=df_takaout[df_takaout['description']==i].sum()['takaout']
    x=abs(x)
    Expense_sum[i]=x

In [31]:
Expense_q=sorted(Expense_sum.values())
Expense_labels=[]
for x in Expense_q:
    for name, number in Expense_sum.items():
        if x == number:
            Expense_labels.append(name)

In [32]:
len(Expense_q)

22

In [33]:
%matplotlib
from matplotlib import font_manager as fm

def my_autopct(pct):
    return ('%.1f' % pct) if pct > 1.76 else ''

#ax.pie(df[col], labels=df.index, autopct=my_autopct, colors=colors)
#label = list(Expense_sum.keys()
#values = list(Expense_sum.values())
explode=[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.02]
colors= cm.rainbow(np.arange(len(Expense_q))/len(Expense_q))
fig, ax = plt.subplots(figsize=(10,5))
ax.pie(Expense_q,labels=Expense_labels,explode=explode,colors=colors,startangle=90,autopct=my_autopct,rotatelabels=False,textprops={"size": 10})
handles, labels = ax.get_legend_handles_labels()


plt.legend(handles[::-1], labels[::-1],bbox_to_anchor=(1.1, 0.95))
plt.show()

Using matplotlib backend: MacOSX


In [39]:
dfHC=df.drop(df[(df.description!='home construction')].index)

In [40]:
dfHC=dfHC.replace('water and s17atftion','water and sanitation')

In [41]:
dfHC['subclass'].unique()

array(['carpentry', 'building parts and materials', 'labour', 'masonry',
       'painting', 'brickbreaking', 'water and sanitation', 'fees',
       'electric'], dtype=object)

In [42]:
HC_Expense=['carpentry', 'building parts and materials', 'labour', 'masonry',
       'painting', 'brickbreaking', 'fees',
       'electric', 'water and sanitation']

In [43]:
HC_Expense_sum={}
for i in HC_Expense:    
    x=dfHC[dfHC['subclass']==i].sum()['takaout']
    x=abs(x)
    HC_Expense_sum[i]=x

In [89]:
HC_quantity=sorted(HC_Expense_sum.values())
HC_labels=[]
for x in HC_quantity:
    for name, number in HC_Expense_sum.items():
        if x == number:
            HC_labels.append(name)

In [127]:
%matplotlib
def my_autopct(pct):
    return ('%.2f' % pct) if pct > 1.765 else ''
color= ['dodgerblue','red','orangered','deepskyblue','limegreen','tomato','firebrick','tomato','deepskyblue']
colors=color[::-1]
fig, ax2 = plt.subplots(figsize=(10,5))
plt.pie(HC_quantity,labels=HC_labels,startangle=90,autopct=my_autopct,labeldistance=1.1,colors =colors,textprops={"size": 10},wedgeprops={'linewidth': 0.4, 'edgecolor':"white"})
handles, labels = ax2.get_legend_handles_labels()
plt.legend(bbox_to_anchor=(1.1, 0.8),labels=HC_labels[::-1],handles=handles[::-1])
plt.show()

Using matplotlib backend: MacOSX


In [107]:
handles

In [108]:
labels

['savings repaid to others',
 'personal grooming',
 'repayment made to diary project',
 'payment to others',
 'vehicle repair costs',
 'toiletries purchased',
 'communications',
 'work tools and materials',
 'jewellery purchases',
 'fees paid',
 'rent paid',
 'real estate purchases',
 'loan or howlat given',
 'education costs',
 'home maintenance',
 'fuel costs',
 'transport costs',
 'clothes, cloth',
 'utilities cost',
 'healthcare costs',
 'recycling stock purchases',
 'snacks, treats, stimulants',
 'farm inputs',
 'gifts to outsiders',
 'other expenditure',
 'food costs',
 'home construction']

In [ ]:
wedgeprops={ 'linewidth' : 0.2, 'edgecolor' : 'white' },fig, ax = plt.subplots(figsize=(10,5)

In [53]:
HC_quantity

[1500, 7500, 9800, 23920, 27700, 35730, 111420, 759900, 2521855]

In [54]:
HC_labels

['electric',
 'fees',
 'brickbreaking',
 'labour',
 'water and sanitation',
 'painting',
 'carpentry',
 'masonry',
 'building parts and materials']

In [60]:
dfHC[dfHC['subclass']=='electric']

,date,ref,code,class,description,subclass,tag,takain,takaout,details,notes,flow,type,quartile
834228,2020-06-09,no rec,27HDM,EHC,home construction,electric,no rec,0,-500,electician,no rec,out,expenditure,1
836637,2020-06-16,no rec,27HDM,EHC,home construction,electric,br,0,-1000,electician,"He do not give Gb kisti to his wife name , rel...",out,expenditure,1


In [27]:
df[df['description']=='savings repaid to others']

,date,ref,code,class,description,subclass,tag,takain,takaout,details,notes,flow,type,quartile
